#### Before You Start


cd .devcontainer </br>
run docker-compose up

In [2]:
! pip install psycopg2
! pip install sqlalchemy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [4]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


Connected Successfully to PostgreSQL server!!


#### View table in python

In [5]:
import pandas as pd

df = pd.read_csv('lap_time.csv')
print(df.head())

   race_id  driver_id  lap  position  time_in_milliseconds
0      841         20    1         1                 98109
1      841         20    2         1                 93006
2      841         20    3         1                 92713
3      841         20    4         1                 92803
4      841         20    5         1                 92342


### Create tables in relational database

In [6]:
try:
    #table_name variable
    create_all_tables_query = '''

    DROP TABLE IF EXISTS lap_time, pit_stop, result, race, driver, constructor, circuit, season CASCADE;

    CREATE TABLE IF NOT EXISTS circuit (
        circuit_id INT PRIMARY KEY,
        -- circuit_ref omitted
        name TEXT,
        location TEXT,
        country TEXT,
        lat float,
        long float,
        alt INT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS constructor (
        constructor_id INT PRIMARY KEY,
        -- removing constructor_ref for normalisation purposes
        name TEXT,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS driver (
        driver_id INT PRIMARY KEY,
        -- removing driver_ref and number for normalisation purposes
        code TEXT,
        first_name TEXT, --no longer forename
        last_name TEXT, --no longer surname
        dob DATE,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS lap_time (
        race_id INT,
        driver_id INT REFERENCES driver,
        lap INT,
        position INT,
        time_in_milliseconds INT,
        PRIMARY KEY(race_id, driver_id, lap)
    );

    CREATE TABLE IF NOT EXISTS pit_stop (
        race_id INT,
        driver_id INT,
        stop INT,
        lap INT,
        time_in_milliseconds INT,
        PRIMARY KEY(race_id, driver_id, lap)
    );

    CREATE TABLE IF NOT EXISTS race (
        race_id INT PRIMARY KEY,
        year INT,
        round INT,
        circuit_id INT REFERENCES circuit,
        name TEXT,
        date DATE,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS result (
        result_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        driver_id INT REFERENCES driver,
        constructor_id INT REFERENCES constructor,
        -- number and grid omitted
        position INT, -- use positionOrder instead of position/positionText
        points INT,
        laps INT,
        -- omit time, unclean
        time_in_milliseconds INT,
        fastest_lap INT,
        rank INT,
        fastest_lap_time_in_milliseconds INT,
        fastest_lap_speed float
    );

    CREATE TABLE IF NOT EXISTS season (
        year INT PRIMARY KEY,
        url TEXT
    );

    '''


    #Execute this command (SQL Query)
    cursor.execute(create_all_tables_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("All tables created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

All tables created successfully in PostgreSQL 


### Create many-to-many tables

In [7]:
try:
    #table_name variable
    create_all_tables_query = '''

    DROP TABLE IF EXISTS race_result, constructor_result CASCADE;

    CREATE TABLE IF NOT EXISTS race_result (
        race_id INT REFERENCES race,
        result_id INT REFERENCES result, 
        PRIMARY KEY (race_id, result_id)
    );

    CREATE TABLE IF NOT EXISTS constructor_result (
        constructor_id INT REFERENCES constructor,
        result_id INT REFERENCES result,
        PRIMARY KEY (constructor_id, result_id)
    );
    '''


    #Execute this command (SQL Query)
    cursor.execute(create_all_tables_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("All tables created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

All tables created successfully in PostgreSQL 


### Check table creation

In [8]:
# [information_schema.tables] keep listing of every table being managed by Postgres for a particular database.
# specifying the tabel_schema to 'public' to only list tables that you create.
cursor.execute("""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_schema = 'public'  
               """)

for table in cursor.fetchall():
    print(table)

('driver',)
('lap_time',)
('pit_stop',)
('circuit',)
('race',)
('result',)
('constructor',)
('season',)
('race_result',)
('constructor_result',)


### Read all CSVs and insert into postgres (requires some waiting)

In [9]:
import pandas as pd
from sqlalchemy import create_engine

# List of CSV files and corresponding table names
csv_files = [
    ("circuit.csv", "circuit"),
    ("constructor.csv", "constructor"),
    ("driver.csv", "driver"),
    ("lap_time.csv", "lap_time"),
    ("pit_stop.csv", "pit_stop"),
    ("race.csv", "race"),
    ("result.csv", "result"),
    ("season.csv", "season")
]

# Connect to the PostgreSQL database
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Loop through the list of CSV files and import them into the PostgreSQL tables
for csv_file, table_name in csv_files:
    try:
        # Load CSV file into DataFrame
        # df = pd.read_csv(csv_file)
        df = pd.read_csv(csv_file, encoding="latin1")

        # Write the DataFrame to the PostgreSQL table
        df.to_sql(table_name, con=engine, if_exists="append", index=False, method="multi", chunksize=100)
        # df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
    except Exception as e:
        print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


Data from circuit.csv successfully imported into circuit!
Data from constructor.csv successfully imported into constructor!
Data from driver.csv successfully imported into driver!
Data from lap_time.csv successfully imported into lap_time!
Data from pit_stop.csv successfully imported into pit_stop!
Data from race.csv successfully imported into race!
Data from result.csv successfully imported into result!
Data from season.csv successfully imported into season!


### Populate many-to-many tables with records

In [10]:
csv_file = 'result.csv'
table_name = 'race_result'

try:
        # Load CSV file into DataFrame
        # df = pd.read_csv(csv_file)
        df_result = pd.read_csv('result.csv', encoding="latin1")
        df_race_result = df_result[['race_id', 'result_id']]
        df_constructor_result = df_result[['constructor_id', 'result_id']]

        # Write the DataFrame to the PostgreSQL table
        df_race_result.to_sql('race_result', con=engine, if_exists="append", index=False, method="multi", chunksize=100)
        # df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


Data from result.csv successfully imported into race_result!


In [11]:
csv_file = 'result.csv'
table_name = 'constructor_result'

try:
        # Load CSV file into DataFrame
        # df = pd.read_csv(csv_file)
        df_result = pd.read_csv('result.csv')
        df_constructor_result = df_result[['constructor_id', 'result_id']]

        # Write the DataFrame to the PostgreSQL table
        df_constructor_result.to_sql('constructor_result', con=engine, if_exists="append", index=False, method="multi", chunksize=100)
        # df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


Data from result.csv successfully imported into constructor_result!


### Query 1: List circuits in the USA

In [12]:
USA_circuits= psql.read_sql("""
                            SELECT *
                            FROM circuit
                            WHERE country = 'USA'
                            """, engine)
display(USA_circuits.style)

,circuit_id,name,location,country,lat,long,alt,url
0,19,Indianapolis Motor Speedway,Indianapolis,USA,39.795000,-86.234700,223,http://en.wikipedia.org/wiki/Indianapolis_Motor_Speedway
1,33,Phoenix street circuit,Phoenix,USA,33.447900,-112.075000,345,http://en.wikipedia.org/wiki/Phoenix_street_circuit
2,37,Detroit Street Circuit,Detroit,USA,42.329800,-83.040100,177,http://en.wikipedia.org/wiki/Detroit_street_circuit
3,42,Fair Park,Dallas,USA,32.777400,-96.758700,139,http://en.wikipedia.org/wiki/Fair_Park
4,43,Long Beach,California,USA,33.765100,-118.189000,12,"http://en.wikipedia.org/wiki/Long_Beach,_California"
5,44,Las Vegas Street Circuit,Nevada,USA,36.116200,-115.174000,639,http://en.wikipedia.org/wiki/Las_Vegas_Street_Circuit
6,46,Watkins Glen,New York State,USA,42.336900,-76.927200,485,http://en.wikipedia.org/wiki/Watkins_Glen_International
7,60,Riverside International Raceway,California,USA,33.937000,-117.273000,470,http://en.wikipedia.org/wiki/Riverside_International_Raceway
8,63,Sebring International Raceway,Florida,USA,27.454700,-81.348300,18,http://en.wikipedia.org/wiki/Sebring_Raceway
9,69,Circuit of the Americas,Austin,USA,30.132800,-97.641100,161,http://en.wikipedia.org/wiki/Circuit_of_the_Americas


### Query 2: List race count of the top 10 drivers in order


In [13]:
driver_race_counts= psql.read_sql("""
                                  SELECT * 
                                  FROM driver AS d
                                  JOIN 
                                    (SELECT driver_id, COUNT(race_id) AS race_count
                                    FROM result
                                    GROUP BY driver_id
                                    ) AS r
                                  ON d.driver_id = r.driver_id
                                  ORDER BY race_count DESC
                                  LIMIT 10
                                """, engine)
display(driver_race_counts.style)

,driver_id,code,first_name,last_name,dob,nationality,url,driver_id,race_count
0,4,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,4,392
1,8,RAI,Kimi,Rï¿½ikkï¿½nen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,8,352
2,1,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,344
3,22,BAR,Rubens,Barrichello,1972-05-23,Brazilian,http://en.wikipedia.org/wiki/Rubens_Barrichello,22,326
4,18,BUT,Jenson,Button,1980-01-19,British,http://en.wikipedia.org/wiki/Jenson_Button,18,309
5,30,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,30,308
6,20,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,300
7,13,MAS,Felipe,Massa,1981-04-25,Brazilian,http://en.wikipedia.org/wiki/Felipe_Massa,13,271
8,815,PER,Sergio,Pï¿½rez,1990-01-26,Mexican,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,815,271
9,119,\N,Riccardo,Patrese,1954-04-17,Italian,http://en.wikipedia.org/wiki/Riccardo_Patrese,119,257


### Query 3: Which driver has the most wins?

In [14]:
driver_win_counts= psql.read_sql("""
                                  SELECT * 
                                  FROM driver AS d
                                  JOIN 
                                    (SELECT driver_id, COUNT(CASE WHEN rank = '1' THEN 1 END) AS first_place_count
                                    FROM result
                                    GROUP BY driver_id
                                    ) AS r
                                  ON d.driver_id = r.driver_id
                                  ORDER BY r.first_place_count DESC
                                  LIMIT 10
                                """, engine)
display(driver_win_counts.style)

,driver_id,code,first_name,last_name,dob,nationality,url,driver_id,first_place_count
0,1,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,66
1,8,RAI,Kimi,Rï¿½ikkï¿½nen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,8,42
2,20,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,38
3,830,VER,Max,Verstappen,1997-09-30,Dutch,http://en.wikipedia.org/wiki/Max_Verstappen,830,32
4,4,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,4,25
5,30,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,30,21
6,3,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,3,20
7,822,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas,822,19
8,17,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(racing_driver),17,19
9,817,RIC,Daniel,Ricciardo,1989-07-01,Australian,http://en.wikipedia.org/wiki/Daniel_Ricciardo,817,16


### Query 4: Which drivers have the same nationality as the constructor?

In [15]:
driver_same_constructor_nationality = psql.read_sql("""
                            SELECT DISTINCT d.first_name, last_name, name AS constructor_name
                            FROM constructor AS c
                            JOIN driver AS d
                            ON d.nationality = c.nationality
                            LIMIT 10
                            """, engine)
display(driver_same_constructor_nationality.style)

,first_name,last_name,constructor_name
0,Bruce,Halford,Toleman
1,Ken,Miles,Brabham-Ford
2,Kevin,Cogan,Sherman
3,Geoff,Crossley,Ferguson
4,Cliff,Allison,Cooper-OSCA
5,Johnny,Claes,ENB
6,Hap,Sharp,Haas F1 Team
7,Alan,Rollinson,Scirocco
8,Nino,Farina,Osella
9,Bill,Aston,Brabham-Climax


### Query 5: Show across time who drove the fastest in the Singapore Grand Prix?

In [16]:
fastest_lap_times_sgp = psql.read_sql("""
                                    SELECT d.first_name, d.last_name, x.time_in_milliseconds, x.date
                                    FROM driver AS d
                                    JOIN (
                                        SELECT l.driver_id, MIN(l.time_in_milliseconds) AS time_in_milliseconds, r.date
                                        FROM lap_time AS l
                                        JOIN race AS r
                                        ON l.race_id = r.race_id
                                        WHERE r.name = 'Singapore Grand Prix'
                                        GROUP BY l.driver_id, r.date
                                    ) AS x
                                    ON d.driver_id = x.driver_id
                                    ORDER BY x.time_in_milliseconds ASC
                                    LIMIT 20
                                      """, engine)
display(fastest_lap_times_sgp.style)

,first_name,last_name,time_in_milliseconds,date
0,Lewis,Hamilton,95867,2023-09-17
1,George,Russell,96273,2023-09-17
2,Fernando,Alonso,96456,2023-09-17
3,Max,Verstappen,96575,2023-09-17
4,Sergio,Pï¿½rez,97108,2023-09-17
5,Alexander,Albon,97342,2023-09-17
6,Carlos,Sainz,97666,2023-09-17
7,Lando,Norris,98046,2023-09-17
8,Valtteri,Bottas,98075,2023-09-17
9,Kevin,Magnussen,98107,2023-09-17


### Query 6: Which constructors have not gotten first place before?

In [17]:
losing_constructors = psql.read_sql("""
                                    SELECT c.constructor_id, c.name
                                    FROM constructor AS c
                                    LEFT JOIN (
                                        SELECT DISTINCT constructor_id
                                        FROM result
                                        WHERE position = 1
                                    ) AS winning_constructors
                                    ON c.constructor_id = winning_constructors.constructor_id
                                    WHERE winning_constructors.constructor_id IS NULL
                                    LIMIT 20
                                      """, engine)
display(losing_constructors.style)

,constructor_id,name
0,7,Toyota
1,8,Super Aguri
2,10,Force India
3,12,Spyker
4,13,MF1
5,14,Spyker MF1
6,15,Sauber
7,16,BAR
8,18,Minardi
9,19,Jaguar


### Query 7: Which races have the tightest outcome?

In [22]:
races_tightest_outcome = psql.read_sql("""
SELECT r.race_id, 
       r.name, 
       (r2.time_in_milliseconds - r1.time_in_milliseconds) AS time_difference_in_miliseconds
FROM result r1
JOIN result r2 ON r1.race_id = r2.race_id
JOIN race r ON r1.race_id = r.race_id
WHERE r1.position = 1 AND r2.position = 2
ORDER BY time_difference_in_miliseconds ASC
LIMIT 1;
""", engine)

display(races_tightest_outcome.style)


,race_id,name,time_difference_in_miliseconds
0,640,Italian Grand Prix,10


### Query 8: Who are the constructors within the top 10 fastest pitstops of all time?

In [19]:
## this might be the most disgusting join i've ever done.

fastest_pit_stop = psql.read_sql("""
                                WITH fastest_pit_stop AS (
                                    SELECT p.race_id, p.driver_id, p.time_in_milliseconds
                                    FROM pit_stop AS p
                                    ORDER BY p.time_in_milliseconds ASC
                                    LIMIT 10
                                )
                                SELECT 
                                    c.name AS constructor_name, 
                                    p.time_in_milliseconds, 
                                    r.date, 
                                    ci.name AS circuit_name
                                FROM constructor AS c
                                JOIN constructor_result AS cr ON c.constructor_id = cr.constructor_id
                                JOIN result AS res ON cr.result_id = res.result_id
                                JOIN fastest_pit_stop AS p ON res.race_id = p.race_id AND res.driver_id = p.driver_id 
                                -- as pit_stop is a weak entity and has composite key
                                JOIN race AS r ON p.race_id = r.race_id
                                JOIN circuit AS ci ON r.circuit_id = ci.circuit_id
                                ORDER BY p.time_in_milliseconds ASC
                                LIMIT 10;
""", engine)

display(fastest_pit_stop.style)


,constructor_name,time_in_milliseconds,date,circuit_name
0,Williams,12897,2011-11-13,Yas Marina Circuit
1,Renault,12959,2011-11-13,Yas Marina Circuit
2,McLaren,13173,2011-07-31,Hungaroring
3,Williams,13186,2011-07-31,Hungaroring
4,Mercedes,13199,2012-07-29,Hungaroring
5,Sauber,13201,2011-07-31,Hungaroring
6,Williams,13206,2012-07-29,Hungaroring
7,Ferrari,13259,2012-05-13,Circuit de Barcelona-Catalunya
8,Williams,13266,2013-05-12,Circuit de Barcelona-Catalunya
9,Red Bull,13335,2012-05-13,Circuit de Barcelona-Catalunya


### Query 9: List all drivers who have driven for Mclaren

In [20]:

all_drivers_mclaren = psql.read_sql(""" SELECT DISTINCT 
    d.first_name,
    d.last_name
FROM driver d
JOIN result r ON d.driver_id = r.driver_id
JOIN constructor c ON r.constructor_id = c.constructor_id
WHERE c.name = 'McLaren';


                                    
                                      """, engine)
display(all_drivers_mclaren.style)

,first_name,last_name
0,Alain,Prost
1,Alexander,Wurz
2,Andrea,de Cesaris
3,Ayrton,Senna
4,Brett,Lunger
5,Brian,Redman
6,Bruce,McLaren
7,Bruno,Giacomelli
8,Carlos,Sainz
9,Daniel,Ricciardo



### Query 10: Which constructor and driver-pairs have the most podium finishes?


In [21]:
driver_const_pair = psql.read_sql("""


SELECT 
    d.first_name || ' ' || d.last_name AS driver_name,
    c.name AS constructor_name,
    COUNT(*) AS podium_finishes
FROM 
    result res
JOIN 
    driver d ON res.driver_id = d.driver_id
JOIN 
    constructor c ON res.constructor_id = c.constructor_id
WHERE 
    res.position IN (1, 2, 3) -- Only podium finishes
GROUP BY 
    d.driver_id, c.constructor_id, d.first_name, d.last_name, c.name
ORDER BY 
    podium_finishes DESC, driver_name, constructor_name
                                  LIMIT 10;

""", engine)
display(driver_const_pair.style)

,driver_name,constructor_name,podium_finishes
0,Lewis Hamilton,Mercedes,150
1,Michael Schumacher,Ferrari,116
2,Max Verstappen,Red Bull,107
3,Sebastian Vettel,Red Bull,65
4,Alain Prost,McLaren,63
5,Valtteri Bottas,Mercedes,58
6,Ayrton Senna,McLaren,55
7,Nico Rosberg,Mercedes,55
8,Rubens Barrichello,Ferrari,55
9,Sebastian Vettel,Ferrari,55
